In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

# Kết nối SQLite
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Câu 1: Tính hệ số tương quan Pearson
cursor.execute("""
CREATE TABLE Scores (
    Day TEXT,
    A REAL,
    B REAL,
    C REAL
)
""")
data = [
    ("Day 1", 8, 9, 7),
    ("Day 2", 7.5, 8.5, 7),
    ("Day 3", 6, 7, 8),
    ("Day 4", 7, 6, 5)
]
cursor.executemany("INSERT INTO Scores VALUES (?, ?, ?, ?)", data)

cursor.execute("""
SELECT
    (COUNT(*) * SUM(A * B) - SUM(A) * SUM(B)) /
    (SQRT(COUNT(*) * SUM(A * A) - SUM(A) * SUM(A)) *
     SQRT(COUNT(*) * SUM(B * B) - SUM(B) * SUM(B))) AS r_ab
FROM Scores
""")
print("Hệ số tương quan Pearson giữa A và B:", cursor.fetchone()[0])


Hệ số tương quan Pearson giữa A và B: 0.6910506641398757


In [2]:
# Câu 2: Kiểm định Chi-Square
import sqlite3
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

# Kết nối SQLite database trong RAM
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Tạo bảng dữ liệu
cursor.execute('''
    CREATE TABLE scores (
        day TEXT,
        model TEXT,
        score REAL
    )
''')

# Dữ liệu gốc
data = [
    ('Day 1', 'A', 8),   ('Day 1', 'B', 9),   ('Day 1', 'C', 7),
    ('Day 2', 'A', 7.5), ('Day 2', 'B', 8.5), ('Day 2', 'C', 7),
    ('Day 3', 'A', 6),   ('Day 3', 'B', 7),   ('Day 3', 'C', 8),
    ('Day 4', 'A', 7),   ('Day 4', 'B', 6),   ('Day 4', 'C', 5)
]

# Chèn dữ liệu vào bảng
cursor.executemany('INSERT INTO scores VALUES (?, ?, ?)', data)
conn.commit()

# Phân loại điểm số
def categorize(score):
    if score <= 6.9:
        return 'Low'
    elif score <= 8:
        return 'Medium'
    else:
        return 'High'

# Lấy dữ liệu ra để xử lý bằng pandas
df = pd.read_sql_query("SELECT * FROM scores", conn)
df['category'] = df['score'].apply(categorize)

# In bảng đã phân loại
print("Dữ liệu đã phân loại")
print(df)

# Tạo bảng tần suất: rows = model, columns = category
contingency_table = pd.crosstab(df['model'], df['category'])

# In bảng tần suất
print("\nBảng tần suất")
print(contingency_table)

# Thực hiện kiểm định Chi-squared
chi2, p, dof, expected = chi2_contingency(contingency_table)

# In kết quả
print("\nKết quả kiểm định χ²")
print(f"Chi-squared: {chi2:.4f}")
print(f"p-value: {p:.4f}")
print(f"Degrees of freedom: {dof}")
print("Expected frequencies:")
print(expected)


Dữ liệu đã phân loại
      day model  score category
0   Day 1     A    8.0   Medium
1   Day 1     B    9.0     High
2   Day 1     C    7.0   Medium
3   Day 2     A    7.5   Medium
4   Day 2     B    8.5     High
5   Day 2     C    7.0   Medium
6   Day 3     A    6.0      Low
7   Day 3     B    7.0   Medium
8   Day 3     C    8.0   Medium
9   Day 4     A    7.0   Medium
10  Day 4     B    6.0      Low
11  Day 4     C    5.0      Low

Bảng tần suất
category  High  Low  Medium
model                      
A            0    1       3
B            2    1       1
C            0    1       3

Kết quả kiểm định χ²
Chi-squared: 5.1429
p-value: 0.2730
Degrees of freedom: 4
Expected frequencies:
[[0.66666667 1.         2.33333333]
 [0.66666667 1.         2.33333333]
 [0.66666667 1.         2.33333333]]


In [ ]:
# Câu 3: Định dạng thời gian
cursor.execute("CREATE TABLE flights (departure_time INTEGER)")
departure_times = [(830,), (1445,), (30,), (1210,), (0,)]
cursor.executemany("INSERT INTO flights VALUES (?)", departure_times)
cursor.execute("""
SELECT
    departure_time,
    printf('%02d:%02d', departure_time / 100, departure_time % 100) AS formatted_time
FROM flights
""")
print("Định dạng giờ:")
for row in cursor.fetchall():
    print(row)

Định dạng giờ:
(830, '08:30')
(1445, '14:45')
(30, '00:30')
(1210, '12:10')
(0, '00:00')


In [4]:
# Câu 4: Phát hiện ngoại lệ với MAD
import sqlite3
import pandas as pd

# Kết nối SQLite trong RAM
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng dữ liệu
cursor.execute('''
    CREATE TABLE scores (
        day TEXT,
        model TEXT,
        score REAL
    )
''')

# Dữ liệu thử nghiệm
data = [
    ('Day 1', 'A', 8),   ('Day 1', 'B', 9),   ('Day 1', 'C', 7),
    ('Day 2', 'A', 7.5), ('Day 2', 'B', 8.5), ('Day 2', 'C', 7),
    ('Day 3', 'A', 6),   ('Day 3', 'B', 7),   ('Day 3', 'C', 8),
    ('Day 4', 'A', 7),   ('Day 4', 'B', 6),   ('Day 4', 'C', 5)
]

# Chèn dữ liệu
cursor.executemany("INSERT INTO scores VALUES (?, ?, ?)", data)
conn.commit()

# Truy vấn SQL tìm các giá trị ngoại lệ dựa vào MAD
query = """
WITH Ordered AS (
    SELECT score,
           ROW_NUMBER() OVER (ORDER BY score) AS rn,
           COUNT(*) OVER () AS cnt
    FROM scores
),
MedianValue AS (
    SELECT AVG(score) AS median
    FROM (
        SELECT score
        FROM Ordered
        WHERE rn = (cnt + 1) / 2
           OR rn = (cnt + 2) / 2
    )
),
Deviations AS (
    SELECT s.score,
           ABS(s.score - m.median) AS abs_dev
    FROM scores s, MedianValue m
),
OrderedDevs AS (
    SELECT abs_dev,
           ROW_NUMBER() OVER (ORDER BY abs_dev) AS rn,
           COUNT(*) OVER () AS cnt
    FROM Deviations
),
MADValue AS (
    SELECT AVG(abs_dev) AS mad
    FROM (
        SELECT abs_dev
        FROM OrderedDevs
        WHERE rn = (cnt + 1) / 2
           OR rn = (cnt + 2) / 2
    )
),
Outliers AS (
    SELECT s.*, m.median, mad.mad,
           ABS(s.score - m.median) AS abs_dev,
           CASE
               WHEN ABS(s.score - m.median) > 1.5 * mad.mad THEN 1
               ELSE 0
           END AS is_outlier
    FROM scores s
    JOIN MedianValue m
    JOIN MADValue mad
)
SELECT * FROM Outliers WHERE is_outlier = 1
"""

# Thực thi truy vấn và in kết quả
outliers = pd.read_sql_query(query, conn)
print(" Các giá trị ngoại lệ theo MAD (1.5x) ")
print(outliers)


 Các giá trị ngoại lệ theo MAD (1.5x) 
     day model  score  median  mad  abs_dev  is_outlier
0  Day 1     B    9.0     7.0  1.0      2.0           1
1  Day 4     C    5.0     7.0  1.0      2.0           1


In [5]:
import sqlite3
import pandas as pd

# Kết nối đến SQLite (trong RAM)
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng Patient
cursor.execute('''
    CREATE TABLE Patient (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        last_name TEXT,
        weight REAL,
        height REAL
    )
''')

# Dữ liệu ví dụ
patients = [
    ('Nguyen', 65, 170),
    ('Nguyen', 64.8, 170),
    ('Tran', 70, 180),
    ('Le', 65, 165),
    ('Nguyen', 80, 175),
    ('Nguyen', 66.5, 170)
]

# Thêm dữ liệu vào bảng
cursor.executemany("INSERT INTO Patient (last_name, weight, height) VALUES (?, ?, ?)", patients)
conn.commit()

# Truy vấn tìm các cặp có thể là cùng một người
query = """
SELECT
    p1.id AS id1, p2.id AS id2,
    p1.last_name AS last_name1, p2.last_name AS last_name2,
    p1.weight AS weight1, p2.weight AS weight2,
    -- Điểm giống nhau
    CASE WHEN p1.last_name = p2.last_name THEN 1 ELSE 0 END AS name_match,
    CASE WHEN ABS(p1.weight - p2.weight) <= 2 THEN 1 ELSE 0 END AS weight_match,
    -- Tổng điểm boolean
    (CASE WHEN p1.last_name = p2.last_name THEN 1 ELSE 0 END +
     CASE WHEN ABS(p1.weight - p2.weight) <= 2 THEN 1 ELSE 0 END) AS total_score,
    -- Đánh dấu nếu đạt đủ 2 điểm
    CASE
        WHEN (p1.last_name = p2.last_name AND ABS(p1.weight - p2.weight) <= 2)
        THEN 'Yes'
        ELSE 'No'
    END AS possibly_same_person
FROM Patient p1
JOIN Patient p2 ON p1.id < p2.id
ORDER BY id1, id2
"""

# Thực hiện truy vấn và hiển thị kết quả
df = pd.read_sql_query(query, conn)

# Chỉ hiển thị những cặp có thể là cùng một người
result = df[df['possibly_same_person'] == 'Yes']

print("Các cặp có thể là cùng một người (dựa trên last_name và weight)")
print(result[['id1', 'id2', 'last_name1', 'weight1', 'last_name2', 'weight2', 'total_score']])


Các cặp có thể là cùng một người (dựa trên last_name và weight)
   id1  id2 last_name1  weight1 last_name2  weight2  total_score
0    1    2     Nguyen     65.0     Nguyen     64.8            2
4    1    6     Nguyen     65.0     Nguyen     66.5            2
8    2    6     Nguyen     64.8     Nguyen     66.5            2
